In [23]:
from pathlib import Path

import tqdm
import pandas as pd
import geopandas as gpd

# 1. Load station name list

In [29]:
from utils.load_data import load_metro_station_list


data_folder = Path.cwd().parent / 'Shanghai'


# Load metro station list
path = data_folder / 'metro_network_shanghai.xlsx'
data = load_metro_station_list(path, time_status='current')
print('\nShape of station list data:', data.shape,
      'Shape of data after removing duplicates:', data.drop_duplicates(subset=['line_name', 'name']).shape)


# Load station loctions
path = data_folder / 'station_location_gcj02_shanghai.csv'
loc = pd.read_csv(path) \
        .assign(lon = lambda x : x['location'].str.split(',').str.get(0).astype(float),
                lat = lambda x : x['location'].str.split(',').str.get(1).astype(float)) \
        [['line_name', 'name', 'lon', 'lat']]

print('\nShape of station location data:', loc.shape,
      'Shape of data after removing duplicates:', data.drop_duplicates(subset=['line_name', 'name']).shape)

# drop duplicates
loc = loc.drop_duplicates(subset=['line_name', 'name'])
# convert into geographical data
loc = gpd.GeoDataFrame(loc, geometry=gpd.points_from_xy(loc['lon'], loc['lat']))


# Merge data
data = data.merge(loc, on=['line_name', 'name'], how='left') \
           .assign(stand_id = lambda x : range(x.shape[0]))
print('Shape of merged data:', data.shape)

Loading data: 22it [00:02, 10.47it/s]


Shape of station list data: (519, 10) Shape of data after removing duplicates: (519, 10)

Shape of station location data: (548, 4) Shape of data after removing duplicates: (519, 10)
Shape of merged data: (519, 14)


In [30]:
import networkx as nx

def graph_add_nodes_from_dataframe(graph, data, node_id_col, attr_list):
    '''
    '''
    
        
    return graph
# =================================================================
def graph_add_edges_from_dataframe(graph, data, node_id_col, attr_list):
    '''
    add edges based raw sequences
    '''
    
    return graph
# =================================================================


def create_line_graph(data):
    '''
    
    '''
    # undirected graph
    graph = nx.Graph()
    
    for line_name, line_station in data.gropuby('line_name'):
            
        line_station = line_station.astype({'no':int}) \
                                   .sort_values('no') \
                                   .reset_index(drop=True)
        
        # number of station
        num_sta = line_station.shape[0]
        # is cycle line
        is_cycle = num_sta['line_cycle'].unique.tolist()
        assert len(is_cycle) == 1
        is_cycle = is_cycle[0]
        
        for ix in range(num_sta - 1):
                
        

,no,name,name_zh,cumulative_distance,station_spacing,opening_date,line_name,line_name_zh,line_color,line_cycle,lon,lat,geometry,stand_id
0,1,Xinzhuang,莘庄,0.000,-,1996-12-28,Line 1,1号线,#E3002B,False,121.385373,31.111152,POINT (121.38537 31.11115),0
1,2,Waihuanlu,外环路,1.303,1.303,1996-12-28,Line 1,1号线,#E3002B,False,121.393020,31.120899,POINT (121.39302 31.12090),1
2,3,Lianhua Road,莲花路,2.762,1.459,1996-12-28,Line 1,1号线,#E3002B,False,121.402910,31.130900,POINT (121.40291 31.13090),2
3,4,Jinjiang Park,锦江乐园,4.395,1.633,1995-04-10,Line 1,1号线,#E3002B,False,121.414107,31.142217,POINT (121.41411 31.14222),3
4,5,Shanghai South Railway Station,上海南站,6.484,2.089,1993-05-28,Line 1,1号线,#E3002B,False,121.430041,31.154579,POINT (121.43004 31.15458),4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514,2,Sanlu Highway,三鲁公路,2.000,2,2018-03-31,Pujiang,浦江线,#B5B5B6,False,121.527389,31.056108,POINT (121.52739 31.05611),514
515,3,Minrui Road,闵瑞路,3.000,1,2018-03-31,Pujiang,浦江线,#B5B5B6,False,121.530323,31.047956,POINT (121.53032 31.04796),515
516,4,Puhang Road,浦航路,3.800,0.8,2018-03-31,Pujiang,浦江线,#B5B5B6,False,121.530591,31.040993,POINT (121.53059 31.04099),516
517,5,Dongchengyi Road,东城一路,5.000,1.2,2018-03-31,Pujiang,浦江线,#B5B5B6,False,121.532093,31.030390,POINT (121.53209 31.03039),517
